# 📘 Documentação das Tabelas Gold

Este documento descreve a modelagem das tabelas criadas na camada **Gold** do Data Lake, a partir das tabelas da camada **Silver**.  

---

## 1. Dimensão Clientes
- **Objetivo:** Criar uma dimensão de clientes com dados únicos e padronizados.  
- **Fonte:** `silver.vendas_clientes`  
- **Campos:**
  - `cliente_id`: Identificador único do cliente.  
  - `Nome_cliente_gold`: Nome do cliente.  
  - `Email_cliente_gold`: Email do cliente.  
  - `Cidade_cliente_gold`: Cidade do cliente.  

---

## 2. Dimensão Produtos
- **Objetivo:** Criar uma dimensão de produtos com informações corrigidas e associar preço unitário.  
- **Fonte:** `silver.vendas_produtos` e `silver.vendas_itens__pedidos`  
- **Campos:**
  - `produto_id`: Identificador único do produto.  
  - `Nome_produto`: Nome corrigido do produto.  
  - `Categoria_produto`: Categoria corrigida do produto.  
  - `Preco_unitario_produto`: Preço unitário corrigido do produto.  

---

## 3. Dimensão Pedidos
- **Objetivo:** Criar uma dimensão de pedidos sem trazer o campo `cliente_id` (mantido apenas na tabela fato).  
- **Fonte:** `silver.vendas_pedidos` e `silver.vendas_itens__pedidos`  
- **Campos:**
  - `Pedido_id`: Identificador único do pedido.  
  - `Quantidade_pedido`: Quantidade corrigida do pedido.  
  - `Data_pedido`: Data do pedido.  

---

## 4. Fato Itens Pedido
- **Objetivo:** Criar a tabela fato que relaciona clientes, produtos e pedidos, calculando o valor total.  
- **Fonte:** `silver.vendas_itens__pedidos` e `silver.vendas_pedidos`  
- **Campos:**
  - `itens_id`: Identificador único do item.  
  - `cliente_id`: Identificador do cliente associado ao pedido.  
  - `produto_id`: Identificador do produto.  
  - `valor_total`: Valor total calculado (`quantidade_corrigida * preco_unitario_corrigido`).  

---

## 📊 Resumo da Modelagem
- **Dimensões criadas:** Clientes, Produtos, Pedidos.  
- **Fato criada:** Itens Pedido.  
- **Estratégia:**  
  - Dimensões armazenam atributos descritivos e corrigidos.  
  - Fato concentra métricas e relacionamentos entre dimensões.  
  - O modelo segue o padrão **Star Schema**, facilitando análises no Power BI ou outras ferramentas de BI.  


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold;

In [0]:
%sql
--Tabela Dimensão clientes

-- CREATE OR REPLACE TABLE gold.gold_clientes
-- USING DELTA
-- AS
-- SELECT DISTINCT 
--     id AS cliente_id, 
--     nome as Nome_cliente_gold, 
--     email as Email_cliente_gold, 
--     cidade as Cidade_cliente_gold
-- FROM silver.vendas_clientes;

-- --Tabela Dimensão produtos 

-- CREATE OR REPLACE TABLE gold.gold_produtos
-- USING DELTA
-- AS
-- SELECT DISTINCT 
--     r.id AS produto_id, 
--     r.nome_corrigido as Nome_produto, 
--     r.categoria_corrigida as Categoria_produto,
--     i.preco_unitario_corrigido as Preco_unitario_produto

-- FROM silver.vendas_produtos r
-- JOIN silver.vendas_itens__pedidos i
-- ON r.id = i.produto_id;


--Tabela Dimensão pedidos

-- Preciso retirar Clientes_id para dimensão silver

-- CREATE OR REPLACE TABLE gold.gold_pedidos
-- USING DELTA
-- AS
-- SELECT 
--     p.id AS Pedido_id,
--     i.quantidade_corrigida AS Quantidade_pedido,
--     p.data AS Data_pedido
-- FROM silver.vendas_pedidos p
-- JOIN silver.vendas_itens__pedidos i
-- ON p.id = i.pedido_id;

-- Tabela Fato - Itens Pedido


-- CREATE OR REPLACE TABLE gold.gold_itens_pedido
-- USING DELTA 
-- AS
-- SELECT
--     i.id AS itens_id,
--     p.cliente_id,
--     i.produto_id,
--     CAST(i.quantidade_corrigida * i.preco_unitario_corrigido AS DECIMAL(18,2)) AS valor_total
-- FROM silver.vendas_itens__pedidos i
-- JOIN silver.vendas_pedidos p
--     ON i.pedido_id = p.id;

    

In [0]:
%sql
select * from gold.gold_produtos;

Necessário posicionar a coluna quantidade e fazer a tabela fato 